## I) Pré traitement

### 0) Récupération

In [ ]:
db = load_all_sentences();
print('chargement de {} vers dans la db'.format(len(db.keys())))

In [ ]:
from collections import defaultdict
base_artistes = defaultdict(set)
for k,v in db.iteritems():
    base_artistes[v['artistes']].add(k)
artistes = { k:v for k,v in artistes.iteritems() if len(v) > 200 }
print('{} artistes'.format(len(artistes)))

### 1) Exploration

In [ ]:
import nltk
test = "Bonjour, je suis un texte d'exemple pour le cours d'Openclassrooms. Soyez attentifs à ce cours !"

nltk.word_tokenize(test)

#### Tokenization

In [ ]:
tokenizer = nltk.RegexpTokenizer(r'\w+')
tokenizer.tokenize("Bonjour, je suis un texte d'exemple pour le cours d'Openclassrooms. Soyez attentifs à ce cours !")


In [ ]:
tokenizer = nltk.RegexpTokenizer(r'\w+')

def freq_stats_corpora():
    corpora = defaultdict(list)

    # Création d'un corpus de tokens par artiste
    for artiste,sentence_id in artistes.iteritems():
        for sentence_id in sentence_id:
            corpora[artiste] += tokenizer.tokenize(
                db[sentence_id]['text'].decode('utf-8').lower()
            )

    stats, freq = dict(), dict()

    for k, v in corpora.iteritems():
        freq[k] = fq = nltk.FreqDist(v)
        stats[k] = {'total': len(v)}

    return (freq, stats, corpora)

# Récupération des comptages
freq, stats, corpora = freq_stats_corpora()
df = pd.DataFrame.from_dict(stats, orient='index')

# Affichage des fréquences
df.sort(columns='total', ascending=False)
df.plot(kind='bar', color="#f56900", title='Top 50 Rappeurs par nombre de mots')



In [ ]:
def freq_stats_corpora():
    corpora = defaultdict(list)
    for artiste,sentence_ids in artistes.iteritems():
        for sentence_id in sentence_ids:
            corpora[artiste] += tokenizer.tokenize(
                db[sentence_id]['text'].decode('utf-8').lower()
            )

    stats, freq = dict(), dict()

    for k, v in corpora.iteritems():
        freq[k] = fq = nltk.FreqDist(v)
        stats[k] = {'total': len(v), 'unique': len(fq.keys())}

    return (freq, stats, corpora)


### 2) Nettoyage / Normalisation

#### 1) supprimer les stopwords

In [ ]:
# Premièrement, on récupère la fréquence totale de chaque mot sur tout le corpus d'artistes
freq_totale = nltk.Counter()
for k, v in corpora.iteritems():
    freq_totale += freq[k]

# Deuxièmement on décide manière un peu arbitraire du nombre de mots les plus fréquents à supprimer. On pourrait afficher un graphe d'évolution du nombre de mots pour se rendre compte et avoir une meilleure heuristique.
most_freq = zip(*freq2.most_common(100))[0] # stopwords =

# On créé notre set de stopwords final qui cumule ainsi les 100 mots les plus fréquents du corpus ainsi que l'ensemble de stopwords par défaut présent dans la librairie NLTK
sw = set()
sw.update(stopwords)
sw.update(tuple(nltk.corpus.stopwords.words('french')))


In [ ]:
def freq_stats_corpora2(lookup_table=[]):
    corpora = defaultdict(list)
    for artist, block_ids in lt_artists.iteritems():
        for block_id in block_ids:
            tokens = tokenizer.tokenize(db_flat[block_id]['text'].decode('utf-8'))
            corpora[artist] += [w for w in tokens if not w in list(sw)]

    stats, freq = dict(), dict()

    for k, v in corpora.iteritems():
        freq[k] = fq = nltk.FreqDist(v)
        stats[k] = {'total': len(v), 'unique': len(fq.keys())}
    return (freq, stats, corpora)

freq2, stats2, corpora2 = freq_stats_corpora2()


#### 2) Lemmatisation ou racinisation (stemming)

##### Racinisation :

In [ ]:
from nltk.stem.snowball import FrenchStemmer

stemmer = FrenchStemmer()

def freq_stats_corpora3(lookup_table=[]):
    corpora = defaultdict(list)
    for artist, block_ids in lt_artists.iteritems():
        for block_id in block_ids:
            tokens = tokenizer.tokenize(db_flat[block_id]['text'].decode('utf-8').lower())
            corpora[artist] += [stemmer.stem(w) for w in tokens if not w in list(sw)]

    stats, freq = dict(), dict()

    for k, v in corpora.iteritems():
        freq[k] = fq = nltk.FreqDist(v)
        stats[k] = {'total': len(v), 'unique': len(fq.keys())}
    return (freq, stats, corpora)

freq3, stats3, corpora3 = freq_stats_corpora3()
df3 = pd.DataFrame.from_dict(stats3, orient='index').sort(columns='unique', ascending=False)


### TP

#### 1) Exploratory Data Analysis

Quelle est la forme du Dataframe ?

Y a t-il des valeurs manquantes ou des valeurs dupliquées ?

Quelles sont les colonnes qui vont nous intéresser ?

Y a-t-il des données aberrantes ou des incohérences majeures dans les données ?

Y a t-il des tweets anormalement longs / courts ? Peut-on les considérer comme des outliers ?

Quel est le ratio tweet qui parlent de “catastrophes” / tweet normaux ?

En regardant quelques tweets au hasard, peut-on deviner facilement la “target” ?

Peut-on déjà détecter des “patterns” ou des mots clés dans les tweets?

A votre avis quel serait l’accuracy score qu’un humain pourrait obtenir s’il prédisait  les données “à la main” ?

#### 2) Text Processing

Pouvez-vous écrire une fonction qui : tokenize un document, supprime les stopwords, supprime les tokens de moins de 3 lettres ?

Comment peut-on reconstituer le corpus (c'est-à dire un texte avec l’ensemble des documents) ?

Une fois ce corpus constitué, combien de tokens uniques le constitue? Ce nombre vous apparaît-il faible, important, gigantesque ?

Comment réduire ce nombre de tokens uniques, ou autrement dit “comment réduire la taille du vocabulaire” de ce corpus ?

Combien de tokens sont présents une seule fois ? Ces tokens nous seront-ils utiles ?

Appliquer une méthode de stemmatisation ou de lemmatisation peut-elle nous aider à réduire la dimensionnalité du corpus ?

Comment visualiser graphiquement, par un WordCloud par exemple, les tokens les plus présents ?

Pouvez vous appliquer tous les traitements évoqués afin de créer une nouvelle colonne “text” qui serait plus pertinente ?

# II) ?